In [24]:
import pandas as pd
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
df_With_Words=pd.read_csv('./parsed data/table.csv')
df=df_With_Words
if 'is_spam' in df:
    y = df['is_spam'].values # get the labels we want
    del df['is_spam'] # get rid of the class label
    X_pay=df['payload'].values
    
    X=df.values
sc =StandardScaler()

#normalize data
#X=sc.fit_transform(X)
#train-test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8)
Xpay_train, Xpay_test, ypay_train, ypay_test = train_test_split(X_pay, y, test_size=0.8)
print(len(X_test))
# Split X_train again to create validation data
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,test_size=0.2)

47873


# Modeling

## MODEL 1 

In [26]:
%%time
# LeNet example (dumbed down to )
import tensorflow as tf
from tensorflow.contrib import learn
from tensorflow.contrib import layers

# Process vocabulary
vocab_processor = learn.preprocessing.VocabularyProcessor(500)
Xpay_train = np.array(list(vocab_processor.fit_transform(Xpay_train)))
Xpay_test = np.array(list(vocab_processor.transform(Xpay_test)))
n_words = len(vocab_processor.vocabulary_)
print('Total words: %d' % n_words)

Total words: 347304
CPU times: user 56.1 s, sys: 960 ms, total: 57.1 s
Wall time: 58.6 s


In [31]:
def word_rnn_model(x, y):
    """Recurrent neural network model to predict from sequence of words
    to a class."""
    # Convert indexes of words into embeddings.
    # This creates embeddings matrix of [n_words, EMBEDDING_SIZE] and then
    # maps word indexes of the sequence into [batch_size, sequence_length,
    # EMBEDDING_SIZE].
    word_vectors = learn.ops.categorical_variable(x, n_classes=n_words,
      embedding_size=50, name='words')

    # Split into list of embedding per word, while removing doc length dim.
    # word_list results to be a list of tensors [batch_size, EMBEDDING_SIZE].
    word_list = tf.unpack(word_vectors, axis=1)

    # Create a Gated Recurrent Unit cell with hidden size of EMBEDDING_SIZE.
    #cell = tf.nn.rnn_cell.GRUCell(EMBEDDING_SIZE)
    #cell = tf.nn.rnn_cell.BasicLSTMCell(EMBEDDING_SIZE,state_is_tuple=False)
    cell = tf.nn.rnn_cell.LSTMCell(50,state_is_tuple=False)
    #cell = tf.nn.rnn_cell.BasicRNNCell(HIDDEN_SIZE)

    # Create an unrolled Recurrent Neural Networks to length of
    # MAX_DOCUMENT_LENGTH and passes word_list as inputs for each unit.
    per_rnn_output, final_encoding = tf.nn.rnn(cell, word_list, dtype=tf.float32)

    # Given encoding of RNN, take encoding of last step (e.g hidden size of the
    # neural network of last step) and pass it as features for logistic
    # regression over output classes.

    prediction, loss = learn.models.logistic_regression(final_encoding, y)

    # Create a training op.
    train_op = tf.contrib.layers.optimize_loss(
      loss, tf.contrib.framework.get_global_step(),
      optimizer='Adam', learning_rate=0.01)

    return {'class': tf.argmax(prediction, 1), 'prob': prediction}, loss, train_op

In [32]:
%%time
# Build model
classifier = learn.Estimator(model_fn=word_rnn_model)

# Train and predict
classifier.fit(Xpay_train, ypay_train, steps=200)
y_predicted = [p['class'] for p in classifier.predict(Xpay_test, as_iterable=True)]
score = metrics.accuracy_score(ypay_test, y_predicted)
print('Accuracy: {0:f}'.format(score))

TypeError: DataType bool for attr 'T' not in list of allowed values: float32, float64, int32, int64, uint8, int16, int8, uint16, float16

(PURPOSE)

(CODE)

(HYPERTUNE PARAMETER) 

## MODEL 2

In [ ]:
COLUMNS = ["payload_smog_index", "organization", "multipart_count", "kl_divergence_eng_lang", "payload_dale_chall_readability_score",
           "frm_fuzz_set_ratio","payload", "cc"]
LABEL="is_spam"
CATEGORICAL_COLUMNS = ["payload", "cc","organization"]
CONTINUOUS_COLUMNS = ["payload_smog_index","multipart_count","kl_divergence_eng_lang","payload_dale_chall_readability_score", "frm_fuzz_set_ratio"]